In [10]:
import pandas as pd
import numpy as np
import json
from scipy.spatial import KDTree

In [2]:
df = pd.read_csv('ep187_electrodes.csv')

In [3]:
x = []
y = []
z = []
positions = []

In [4]:
for index, row in df.iterrows():
    pos = json.loads(row.position)
    positions.append(pos)
    x.append(pos[0])
    y.append(pos[1])
    z.append(pos[2])

In [5]:
df['x'] = x
df['y'] = y
df['z'] = z

In [6]:
df2 = pd.read_csv('brain.csv')

In [7]:
df2 = df2.rename(columns={'Points:0': 'x', 'Points:1': 'y', 'Points:2': 'z' })
df2

,Normals:0,Normals:1,Normals:2,x,y,z
0,0.098496,-0.357720,-0.92862,99.650,88.247,41.569
1,-0.065635,-0.223420,-0.97251,98.919,88.758,41.425
2,0.138640,-0.136590,-0.98088,99.702,88.788,41.447
3,-0.234910,-0.157350,-0.95920,98.091,89.319,41.429
4,-0.005564,-0.004576,-0.99997,98.757,89.492,41.338
...,...,...,...,...,...,...
127283,0.094383,-0.181240,0.97890,105.600,103.570,195.610
127284,0.053224,-0.084879,0.99497,105.590,104.140,195.680
127285,0.027702,-0.034398,0.99902,105.610,104.870,195.720
127286,0.008826,0.014855,0.99985,105.640,105.580,195.730


In [8]:
allPoints = []
for index, row in df2.iterrows():
    point = [row.x, row.y, row.z]
    allPoints.append(point)

In [11]:
NDIM = 3 # number of dimensions

# read points into array
a = np.array(allPoints)
# print(a)
# find 10 nearest points
tree = KDTree(a, leafsize=a.shape[0]+1)

newPoints = []
newX = []
newY = []
newZ = []

for position in positions:
    point = np.array(position)
    distances, ndx = tree.query([point], k=1)
    nearPoint = [a[ndx][0][0], a[ndx][0][1], a[ndx][0][2]]
    newPoints.append(nearPoint)
    newX.append(a[ndx][0][0])
    newY.append(a[ndx][0][1])
    newZ.append(a[ndx][0][2])

In [12]:
df['newX'] = newX
df['newY'] = newY
df['newZ'] = newZ
df['newPosition'] = newPoints

In [14]:
df.head()

,electrode_number,E_EEG,E_Brain,position,position_marker,x,y,z,newX,newY,newZ,newPosition
0,11,E11,PS1,"[138.115783691406,166.815368652344,110.1925659...",Y,138.115784,166.815369,110.192566,138.58,164.07,116.200,"[138.58, 164.07, 116.2]"
1,12,E12,PS2,"[142.168121337891,181.290649414063,112.6060028...",Y,142.168121,181.290649,112.606003,136.71,176.99,121.100,"[136.71, 176.99, 121.1]"
2,13,E13,PS3,"[141.510070800781,183.282073974609,121.5120697...",Y,141.510071,183.282074,121.512070,137.84,179.49,124.130,"[137.84, 179.49, 124.13]"
3,19,E19,AT1,"[134.525161743164,137.630401611328,79.41387939...",Y,134.525162,137.630402,79.413879,134.68,137.82,78.492,"[134.68, 137.82, 78.492]"
4,20,E20,AT2,"[139.903259277344,145.456604003906,83.08189392...",Y,139.903259,145.456604,83.081894,140.06,145.14,84.123,"[140.06, 145.14, 84.123]"


In [15]:
df.to_csv('ep187_electrodes_new.csv', index=False)